Importar librerias

In [1]:
import pandas as pd
import requests
import io
import re
from datetime import datetime

Lectura del archivo YELP bussines.pkl

In [2]:
def cargar_dataset_pickle(file_url):
    response = requests.get(file_url)
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
    # Cargar el contenido en un DataFrame de Pandas
        df = pd.read_pickle(io.BytesIO(response.content))
        return df
    else:
        print(f'Error al obtener el archivo. Código de estado: {response.status_code}')

url='https://drive.usercontent.google.com/download?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu&export=download&authuser=0&confirm=t&uuid=189d48ef-ef64-4bc9-aec7-9e654cb4c757&at=APZUnTVmlezTicVw58BDGG9siM6Q:1704808512976'
dfbusinessYelp=cargar_dataset_pickle(url)

El alcance de nuestro proyecto se limita al negocio de ulta beauty , por lo que se va a filtrar

In [3]:
#Quitamos las columnas duplicadas
dfbusinessYelp = dfbusinessYelp.loc[:, ~dfbusinessYelp.columns.duplicated()]
#filtramos ulta_beauty 
dfbusinessYelp = dfbusinessYelp[dfbusinessYelp['name'] == 'Ulta Beauty']

Solo nos interesan los locales que están en funcionamiento por los que son is_open = 0 no se van a tomar en cuenta

In [4]:
#dfbusinessYelp = dfbusinessYelp[dfbusinessYelp['is_open'] == 1]
#borramos la columna is_open
dfbusinessYelp = dfbusinessYelp.drop('is_open', axis=1)

La dirección que se va a utilizar es la de latitude y longitude por lo que también borramos la columna address

In [5]:
dfbusinessYelp = dfbusinessYelp.drop('address', axis=1)

La columna de name también se va a eliminar ya que todos los locales se llaman igual y solo se van a dejar los bussiness_id para identificarlos

In [6]:
dfbusinessYelp = dfbusinessYelp.drop('name', axis=1)

Borramos la columna de categories ya que todos los lugares de ultabeauty estan relacionados con el area de la belleza y esta columna es irrelevante

In [7]:
dfbusinessYelp = dfbusinessYelp.drop('categories', axis=1)

Expandemos la columna attributes

In [8]:
# Función para expandir el diccionario en la columna 'attributes'
def expandir_attributes(row):
    if row['attributes']:
        return pd.Series(row['attributes'])
    else:
        return pd.Series()

# Aplicar la función a cada fila del DataFrame
df_expandido = dfbusinessYelp.apply(expandir_attributes, axis=1)

# Concatenar el DataFrame original con el nuevo DataFrame expandido
df_resultado = pd.concat([dfbusinessYelp, df_expandido], axis=1)

# Eliminar la columna original 'attributes' si es necesario
dfbusinessYelp = df_resultado.drop('attributes', axis=1)

Expandemos la columna hour

In [9]:
# Función para expandir el diccionario en la columna 'hour' con prefijo 'hour'
def expandir_hour(row):
    if row['hours']:
        hour_dict = row['hours']
        expanded_dict = {}
        for day, time_range in hour_dict.items():
            expanded_dict[f'hours_{day}'] = time_range
        return pd.Series(expanded_dict)
    else:
        return pd.Series()

# Aplicar la función a cada fila del DataFrame
df_expandido_hour = dfbusinessYelp.apply(expandir_hour, axis=1)

# Concatenar el DataFrame original con el nuevo DataFrame expandido
df_resultado = pd.concat([dfbusinessYelp, df_expandido_hour], axis=1)

# Eliminar la columna original 'hour' si es necesario
dfbusinessYelp = df_resultado.drop('hours', axis=1)

Valores nulos

In [10]:
round(dfbusinessYelp.isnull().sum() / dfbusinessYelp.shape[0] * 100, 2).astype(str) + ' %'


business_id                     0.0 %
city                            0.0 %
state                           0.0 %
postal_code                     0.0 %
latitude                        0.0 %
longitude                       0.0 %
stars                           0.0 %
review_count                    0.0 %
BikeParking                   27.87 %
BusinessAcceptsCreditCards    18.03 %
BusinessParking               19.67 %
ByAppointmentOnly             19.67 %
DogsAllowed                   95.08 %
GoodForKids                   47.54 %
RestaurantsPriceRange2        16.39 %
WheelchairAccessible          96.72 %
WiFi                          98.36 %
hours_Monday                    0.0 %
hours_Tuesday                   0.0 %
hours_Wednesday                 0.0 %
hours_Thursday                  0.0 %
hours_Friday                    0.0 %
hours_Saturday                  0.0 %
hours_Sunday                    0.0 %
dtype: object

Las columnas DogsAllowed , WheelchairAccessible ,WiFi,GoodForKids contienen gran cantidad  de valores nulos por lo que serán elimnadas

In [11]:
dfbusinessYelp.drop(columns=['DogsAllowed','WheelchairAccessible','WiFi','GoodForKids'],inplace=True)

Los valores nulos de las columnas BikeParking , BusinessAcceptsCreditCards , ByAppointmentOnly serán reemplazados por False ya que solo contiene valores True y False

In [12]:
dfbusinessYelp[['BikeParking', 'BusinessAcceptsCreditCards', 'ByAppointmentOnly']] = dfbusinessYelp[['BikeParking', 'BusinessAcceptsCreditCards', 'ByAppointmentOnly']].fillna('False')

Observando los datos de la columna 'BusinessParking', notamos que es un diccionario con claves como 'garage', 'street', 'validated', 'lot', y 'valet'. Al analizar los valores asociados a estas claves, observamos que la clave 'garage' tiene todos sus valores establecidos como False. La clave 'street' presenta 45 valores como False y solo 1 como True. La clave 'validated' muestra todos sus valores como False. En cuanto a 'lot', hay 28 valores True y 18 False, y 'valet' contiene exclusivamente valores False.

Dada la prevalencia de valores False en la mayoría de las claves, hemos decidido eliminar por completo la columna 'BusinessParking' 

In [13]:
dfbusinessYelp['BusinessParking'].value_counts()

BusinessParking
{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}     28
{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}    14
None                                                                                     5
{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}      2
Name: count, dtype: int64

In [14]:
dfbusinessYelp = dfbusinessYelp.drop('BusinessParking', axis=1)

Eliminamos la columna RestaurantsPriceRange2 ya que solo nos enfocamos en los productos relacionados con la belleza

In [15]:
dfbusinessYelp = dfbusinessYelp.drop('RestaurantsPriceRange2', axis=1)

Normalización del nombre de las columnas

Todos los nombres de las columnas deben estar en minusculas y las palabras deben estar separadas con '_' ya que estaremos utilizando el formato de snake_case

In [16]:
# Función para convertir el nombre de columna a formato snake_case
def snake_case(column_name):
    return re.sub(r'(?<=[a-z])(?=[A-Z])', '_', column_name).lower()

# Aplicar la función a todos los nombres de columnas
dfbusinessYelp.columns = dfbusinessYelp.columns.map(snake_case)

Nomralización de los datos dentro de las filas

Para la columna city convertiremos a snake_case

In [17]:
dfbusinessYelp['city'] = dfbusinessYelp['city'].str.lower().str.replace(' ', '_')

Para la columna state , se va a reemplazar las siglas por el nombre de los estados

In [18]:
estado_mapping = {
    'PA': 'pennsylvania',
    'FL': 'florida',
    'NV': 'nevada',
    'LA': 'louisiana',
    'AZ': 'arizona',
    'AB': 'alberta',
    'IN': 'indiana',
    'TN': 'tennessee',
    'MO': 'missouri',
    'CA': 'california',
    'ID': 'idaho',
    'NJ': 'new_jersey',
    'DE': 'delaware',
    'IL': 'illinois'
}

# Aplicar el mapeo a la columna 'state'
dfbusinessYelp['state'] = dfbusinessYelp['state'].map(estado_mapping)


convertimos a formato hora los horarios de las columnas que tienen hours

In [19]:
# Función para convertir el formato de horas
def convertir_formato_horas(rango_horas):
    hora_inicio, hora_fin = map(lambda x: datetime.strptime(x, "%H:%M"), rango_horas.split('-'))
    return f"{hora_inicio.strftime('%H:%M')} - {hora_fin.strftime('%H:%M')}"

# Obtener todas las columnas de días de la semana
columnas_dias_semana = [col for col in dfbusinessYelp.columns if col.startswith('hours_')]

# Aplicar la función a todas las columnas de días de la semana
for columna in columnas_dias_semana:
    dfbusinessYelp[columna] = dfbusinessYelp[columna].apply(convertir_formato_horas)


In [20]:
dfbusinessYelp.head()

,business_id,city,state,postal_code,latitude,longitude,stars,review_count,bike_parking,business_accepts_credit_cards,by_appointment_only,hours_monday,hours_tuesday,hours_wednesday,hours_thursday,hours_friday,hours_saturday,hours_sunday
883,4uqRhXZTOzKF2ZhxbWzxfA,newark,pennsylvania,19702,39.672058,-75.6489,3.5,11,False,True,True,10:00 - 20:00,10:00 - 20:00,10:00 - 20:00,10:00 - 21:00,10:00 - 18:00,10:00 - 18:00,10:00 - 20:00
1063,fWMPbickerGWohPy2vDL5A,plainfield,arizona,46168,39.713441,-86.357947,3.0,14,True,True,False,10:00 - 20:00,10:00 - 20:00,10:00 - 20:00,10:00 - 21:00,10:00 - 18:00,10:00 - 18:00,10:00 - 20:00
5488,DJZQCN0NUej_EtviN4rUlg,philadelphia,pennsylvania,19131,39.978981,-75.27146,3.5,12,True,True,False,09:00 - 20:00,09:00 - 20:00,09:00 - 20:00,09:00 - 20:00,09:00 - 20:00,09:00 - 20:00,11:00 - 18:00
13384,Vxqa8u_5RD5e7oBqdaU0yQ,fairview_heights,alberta,62208,38.596645,-89.987348,3.5,13,True,True,False,10:00 - 20:00,10:00 - 20:00,10:00 - 20:00,10:00 - 21:00,10:00 - 18:00,10:00 - 18:00,10:00 - 20:00
13760,idP674ti6a8yg8z2xFcCgA,newtown_square,arizona,19073,39.987189,-75.403201,2.5,24,False,True,False,10:00 - 20:00,10:00 - 20:00,10:00 - 20:00,10:00 - 21:00,10:00 - 18:00,10:00 - 18:00,10:00 - 20:00


In [21]:
dfbusinessYelp.columns

Index(['business_id', 'city', 'state', 'postal_code', 'latitude', 'longitude',
       'stars', 'review_count', 'bike_parking',
       'business_accepts_credit_cards', 'by_appointment_only', 'hours_monday',
       'hours_tuesday', 'hours_wednesday', 'hours_thursday', 'hours_friday',
       'hours_saturday', 'hours_sunday'],
      dtype='object')

In [ ]:
#Agregar source
#Corregir AB
#Quitar is_open